# Running ImageJ via Python

The idea behind the code below is to be able to run my ImageJ code from Python. Not sure if it will work.

In [ ]:
import imagej
import scyjava as sj

# initialize imagej
ij = imagej.init('sc.fiji:fiji:2.3.1', mode='interactive')

### Grouping all of the csv files

The code below should be run after you run the Endplate-analysis.ijm script.

In [27]:
import os
import numpy as np
import pandas as pd

def group_func(s):
    """Add Group Info based on Mouse #"""
    if int(s) in [7488, 8834, 8835, 7482, 7485, 8831]:
        group = "Crush"
    elif int(s) in [8593, 8836, 8837, 8591, 8832, 8833]:
        group = "Sham"
    elif int(s) in [8594, 8595, 7591, 7592, 8592]:
        group = "Tetanus"
    elif int(s) in [123, 144, 151, 125, 135, 152]:
        group = "Tetanus-Crush"
    else:
        group = "Other"
    
    return group

def percent_func(s):
    '''Adjusting values that are over 100% down to 100%'''
    if s > 1:
        val = 100
    else:
        val = s*100
    return val

def muscle_group(s):
    '''Adjusting the groups to be the same'''
    if s in ['GAS','MG','LG']:
        val = 'LG-MG'
    else:
        val = s
    return val

In [28]:
# Grabbing the files
path = r'G:\Tetanus\NMJ Data\raw images\Completed\Analysis_Results'
dir_list = os.listdir(path)

filelist = [file for file in dir_list if file.endswith(".csv")]

In [29]:
# Creating the dataframe
df = pd.DataFrame()
for file in filelist:
    titlesplit = file.split('_')
    
    df1 = pd.read_csv(os.path.join(path,file),index_col=0)
    df1.dropna(axis=0, how='all', inplace=True)
    
    titles = {'Mouse':[titlesplit[0] for i in range(len(df1))],
              'Muscle':[titlesplit[1][:-1].upper() for i in range(len(df1))]}
    
    df1=df1.assign(**titles)
    
    df = pd.concat([df,df1], ignore_index=True)

df["Group"] = df["Mouse"].apply(group_func)
df["Muscle"] = df["Muscle"].apply(muscle_group)

In [30]:
df.head()

,varea0,varea1,varea2,barea1,barea2,tarea0,tarea1,marea1,marea2,vcov01,...,vcov12,vcov22,tcov01,tcov02,tcov12,mcov11,mcov22,Mouse,Muscle,Group
0,24.324,17.876,8.914,9.009,3.793,25.036,18.966,14.889,6.638,2.700,...,4.712,2.350,2.779,6.600,5.000,1.653,1.750,123,TA,Tetanus-Crush
1,40.809,39.890,34.375,32.108,25.429,40.809,40.380,36.275,31.311,1.271,...,1.569,1.352,1.271,1.605,1.588,1.130,1.231,123,TA,Tetanus-Crush
2,42.123,37.811,27.944,29.934,24.295,43.035,40.381,33.416,24.959,1.407,...,1.556,1.150,1.438,1.771,1.662,1.116,1.027,123,TA,Tetanus-Crush
3,36.000,30.583,14.083,5.417,0.583,36.583,33.917,25.667,9.250,6.646,...,52.429,24.143,6.754,62.714,58.143,4.738,15.857,123,TA,Tetanus-Crush
4,36.765,31.209,22.277,27.723,23.039,38.072,33.987,28.922,20.153,1.326,...,1.355,0.967,1.373,1.652,1.475,1.043,0.875,123,TA,Tetanus-Crush


In [31]:
for title in ["vcov01", "vcov02", "vcov11", "vcov12", "vcov22", "tcov01", "tcov02", "tcov12", "mcov11", "mcov22"]:
    df[title] = df[title].apply(percent_func)

df.head()

,varea0,varea1,varea2,barea1,barea2,tarea0,tarea1,marea1,marea2,vcov01,...,vcov12,vcov22,tcov01,tcov02,tcov12,mcov11,mcov22,Mouse,Muscle,Group
0,24.324,17.876,8.914,9.009,3.793,25.036,18.966,14.889,6.638,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,123,TA,Tetanus-Crush
1,40.809,39.890,34.375,32.108,25.429,40.809,40.380,36.275,31.311,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,123,TA,Tetanus-Crush
2,42.123,37.811,27.944,29.934,24.295,43.035,40.381,33.416,24.959,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,123,TA,Tetanus-Crush
3,36.000,30.583,14.083,5.417,0.583,36.583,33.917,25.667,9.250,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,123,TA,Tetanus-Crush
4,36.765,31.209,22.277,27.723,23.039,38.072,33.987,28.922,20.153,100.0,...,100.0,96.7,100.0,100.0,100.0,100.0,87.5,123,TA,Tetanus-Crush


In [32]:
grouped = df.groupby("Group")
for name, group in grouped:
    group.sort_values(["Muscle","Mouse"]).to_csv(os.path.join(path,name+"_Data.csv"), index=False)

In [33]:
mus_ave = df.groupby(["Group", "Muscle"]).mean()
mus_ave.to_csv(os.path.join(path,"Muscle_Average.csv"))
mus_ave

varea0     varea1     varea2     barea1     barea2  \
Group         Muscle                                                          
Crush         LG-MG   38.448525  33.277939  25.278667  32.986488  25.258781   
              SOL     42.689903  38.427389  30.105752  35.710575  25.876690   
              TA      35.840117  32.810160  25.767738  31.581427  23.936087   
Other         TA      37.688526  28.372500  15.641750  33.001250  24.415289   
Sham          LG-MG   33.398674  28.162539  20.850760  31.167824  24.935613   
              SOL     41.133513  36.353597  28.385345  37.037580  28.758050   
              TA      32.072236  27.945347  21.684125  30.126293  23.789054   
Tetanus       LG-MG   30.129399  25.925490  17.878864  28.772390  23.254964   
              TA      29.793350  26.806133  21.148720  28.311963  22.408127   
Tetanus-Crush LG-MG   40.570156  31.660296  17.228640  30.457328  23.842441   
              TA      34.309189  29.587137  20.975978  29.769288  22.646822   

                         tarea0     tarea1     marea1     marea2      vcov01  \
Group         Muscle                                                           
Crush         LG-MG   38.769091  33.558215  26.106929  18.772276   98.906061   
              SOL     42.573062  40.220770  20.134027  12.049575  100.000000   
              TA      35.678121  32.608903  25.344461  17.587495   99.536408   
Other         TA      38.051776  28.454053  23.456868  12.238908   96.911842   
Sham          LG-MG   33.248711  27.428426  21.505061  14.207258   97.747697   
              SOL     41.136756  36.926059  28.592462  19.469370   99.375630   
              TA      31.984957  27.793145  21.399338  14.380034   98.588352   
Tetanus       LG-MG   30.030834  25.683510  19.358627  10.557315   99.056818   
              TA      29.657327  26.185297  18.726613  12.394217   98.715667   
Tetanus-Crush LG-MG   36.836849  27.780419  24.091500  11.971156   99.175806   
              TA      34.302358  29.521464  23.855553  15.298186   98.084097   

                          vcov02     vcov11      vcov12     vcov22     tcov01  \
Group         Muscle                                                            
Crush         LG-MG    99.440404  90.850842   95.517508  81.717508  99.071717   
              SOL     100.000000  98.486726  100.000000  96.666372  99.934513   
              TA       99.762136  94.948544   98.344660  87.740291  99.063107   
Other         TA       98.319737  78.767105   87.682895  60.053947  96.956579   
Sham          LG-MG    98.989309  85.613816   91.750493  73.004605  96.797862   
              SOL      99.784034  91.524370   96.427731  85.615966  99.376471   
              TA       99.393466  87.516193   91.923864  76.803409  97.942614   
Tetanus       LG-MG    99.752597  87.273052   92.769156  70.458442  98.589935   
              TA       99.366000  90.826667   94.919667  80.865667  98.114667   
Tetanus-Crush LG-MG    99.806989  82.523656   89.153226  62.845699  99.238172   
              TA       99.199461  88.173585   93.004313  74.885714  97.395418   

                          tcov02     tcov12     mcov11     mcov22  
Group         Muscle                                               
Crush         LG-MG    99.608418  94.110438  74.802357  65.416835  
              SOL     100.000000  99.932743  54.485841  43.946903  
              TA       99.521359  96.737379  77.623786  68.553883  
Other         TA       98.038158  85.115789  66.306579  46.414474  
Sham          LG-MG    98.288322  88.490461  68.179112  54.590954  
              SOL      99.842017  94.975630  75.105882  63.056303  
              TA       99.040625  90.021023  69.013068  55.711648  
Tetanus       LG-MG    99.337662  90.838961  66.259740  44.200000  
              TA       98.911667  92.594667  64.025000  51.312333  
Tetanus-Crush LG-MG    99.639247  88.887097  62.905914  43.922043  
              TA       98.594070  91.988140  76.287062  62.724798

In [34]:
mouse_ave = df.groupby(["Group", "Muscle", "Mouse"]).mean()
mouse_ave.to_csv(os.path.join(path,"Mouse_Average.csv"))
mouse_ave

varea0     varea1     varea2     barea1  \
Group         Muscle Mouse                                               
Crush         LG-MG  7482   41.273095  37.018952  27.034762  33.218238   
                     7485   39.897926  29.633037  17.639741  33.824370   
                     7488   44.536250  43.691675  39.588475  35.482950   
                     8831   38.439759  33.013241  25.392456  32.083570   
                     8834   38.173802  30.883163  21.532047  33.903221   
                     8835   31.229432  29.418023  23.237773  29.921568   
              SOL    8831   41.017397  37.011544  29.129912  34.751015   
                     8834   45.217244  40.566889  31.580356  37.160578   
              TA     7482   36.139047  33.938047  27.315372  31.625674   
                     7485   37.614200  32.185720  21.940920  31.678360   
                     7488   33.033030  32.207909  28.632788  29.930576   
                     8831   33.301571  30.385762  23.589190  29.061810   
                     8834   38.460318  32.853568  23.429932  32.959818   
                     8835   35.176325  33.709875  27.847450  32.641800   
Other         TA     173    37.688526  28.372500  15.641750  33.001250   
Sham          LG-MG  8591   33.472873  32.243127  27.556228  30.682519   
                     8593   38.104754  34.669211  26.765816  34.587737   
                     8832   32.364354  26.693969  15.398708  31.346708   
                     8833   25.617355  24.466009  20.564542  25.218514   
                     8836   31.416887  19.764472  11.230085  31.258774   
                     8837   37.541350  30.476803  22.316102  33.093204   
              SOL    8591   44.805000  42.341763  35.353026  39.794632   
                     8836   35.241667  28.462490  20.697725  33.590961   
                     8837   46.499100  42.183467  32.628567  39.404567   
              TA     8591   28.948444  28.113175  25.336683  26.281333   
                     8593   34.970952  32.920286  25.354381  32.288365   
                     8833   36.971443  36.811885  35.059820  35.087197   
                     8836   28.409797  20.908820  12.560662  27.912805   
                     8837   38.398250  30.164250  19.689312  33.182563   
Tetanus       LG-MG  7591   26.663976  20.455690  12.760071  26.065405   
                     7592   29.493895  26.227250  17.364145  28.566118   
                     8592   26.966619  24.951562  19.840638  26.059981   
                     8595   36.316906  29.561494  18.445000  33.645012   
              TA     7592   29.144210  24.563355  17.672194  28.319065   
                     8592   30.009574  28.698000  25.384689  28.735115   
                     8594   29.382564  27.751040  22.699327  27.555089   
                     8595   30.695276  25.861566  18.524237  28.972382   
Tetanus-Crush LG-MG  125    34.837850  28.505125  17.634275  28.602800   
                     135    40.373031  32.492375  17.321094  31.766969   
                     151    47.331859  34.161676  15.308746  32.368113   
                     152    34.884558  29.845930  19.952558  28.052837   
              TA     123    31.554011  24.132064  14.275957  27.603670   
                     135    40.524930  37.010035  26.462526  33.307421   
                     144    30.488620  23.300013  12.845570  27.253367   
                     151    33.892586  32.255857  26.312457  29.612429   
                     152    37.628577  35.214521  29.226930  32.750028   

                               barea2     tarea0     tarea1     marea1  \
Group         Muscle Mouse                                               
Crush         LG-MG  7482   25.032000  41.258524  35.677095  34.266714   
                     7485   25.315963  39.745333  28.513852  17.259667   
                     7488   25.779050  44.592300  43.992450  39.567850   
                     8831   25.079241  38.353797  33.241684  24.770076   
                     8834   25.665